
# Input data

## Weather forcing

The first example uses data from ECMWF ERA5 obtained from [CDS](https://cds.climate.copernicus.eu/#!/home)

A common format for multidimensional data is [NetCDF](https://www.unidata.ucar.edu/software/netcdf/).

An excellent python package for working with NetCDF in Python is [Xarray](http://xarray.pydata.org/en/stable/).

To work with NetCDF data first we need to install the `netcdf4` and `xarray` packages

In [ ]:
! pip install netcdf4 xarray

Opening a NetCDF file with Xarray returns an `xarray.Dataset`

In [ ]:
import xarray as xr

ds = xr.open_dataset("data/era5_small.nc")
ds

One of the datam variables is the t2m (Air temperature at 2m above the ground) which we can access in two different ways:

1. Using the string "t2m"
2. Or as a property of the Dataset object `.t2m`

In [ ]:
ds["t2m"]

In [ ]:
ds.t2m

To select the first timestep, we can use the `xarray.Dataset.isel` method.

In [ ]:
ds0 = ds.isel(time=0)
ds0

The dataset from the first timestep is also a Dataset, except it no longer has a time dimension.

To plot a map of the air temperature we can use select the `t2m` property and call the plot method.

In [ ]:
ds0.t2m.plot()

## Pressure

Atmospheric pressure comes in different flavors.

`Surface pressure` (sp) is the pressure at ground level, i.e. following the terrain.

In the following plot you can clearly see the decrease in surface pressure with altitude in areas with mountains.

This is *NOT* the variable used by MIKE 21 FM HD.

In [ ]:
ds0.sp.plot()

MIKE 21 FM HD expects the atmospheric pressure reduced to Mean Sea Level. (`msl`) 

In the following plot the pressure field has a much smaller range, since the variation with altitude have been removed.

In [ ]:
ds0.msl.plot()

## Wind

The wind velocity components U,V at 10m above the ground, are found in the two data variables, `u10` and `v10`

In [ ]:
ds0.u10.long_name

In [ ]:
ds0.v10.long_name

## Convert to dfs2

### Sub-region

In this case the NetCDF file covers a much larger area, than is required for our modelling work, so first step is to create a spatial subset to match our Area of Interest.

Since the latitude dimension is ordered in decreasing order in this dataset, the slice has to go from high to low.

In [ ]:
ds_aoi = ds.sel(longitude=slice(10,30), 
                latitude=slice(60,45)) # N -> S
ds_aoi

Make a plot to check that the area is the one we expect.

In [ ]:
ds_aoi.isel(time=0).u10.plot(cmap='jet')

### Time

Now we are ready to convert this data to dfs2.

First step is to create a time axis, which can be understood by MIKE IO.

In [ ]:
import pandas as pd

time = pd.DatetimeIndex(ds.time)
time

The second step is to collect the variables that we need, and at the same time do unit conversion where it is necessary. In this case for pressure and temperature.

In [ ]:
data = [ds_aoi.msl.values / 100.0, # Pa -> hPa
        ds_aoi.u10.values,
        ds_aoi.v10.values,
        ds_aoi.t2m.values - 273.15] # K -> °C

The third step is to create the proper EUM item information for each variable.

In [ ]:
from mikeio import EUMType, EUMUnit, ItemInfo

items = [
        ItemInfo("MSLP", EUMType.Pressure, EUMUnit.hectopascal),
        ItemInfo("U 10m", EUMType.Wind_Velocity, EUMUnit.meter_per_sec),
        ItemInfo("V 10m", EUMType.Wind_Velocity, EUMUnit.meter_per_sec),
        ItemInfo("Temperature", EUMType.Temperature, EUMUnit.degree_Celsius)
]

The fourth step is to create the `mikeio.Grid2D` geometry. 

In [ ]:
import mikeio

g = mikeio.Grid2D(x=ds_aoi.longitude, y=ds_aoi.latitude[::-1], projection="LONG/LAT")
g

The fifth step is to gather the data, time, item and geometry information in a `mikeio.Dataset`. 
And remember to flip the dataset upside-down since the y-axis in MIKE is south-to-north. 

In [ ]:
my_ds = mikeio.Dataset(data=data, time=time, items=items, geometry=g).flipud()
my_ds

Let's plot to confirm that 

In [ ]:
my_ds.U_10m.plot();

The dimensions for at  DFS2 are expected to be (t, y, x), which matches the ones used by this dataset.

This convention is recommended by the [CF convention](http://cfconventions.org/cf-conventions/cf-conventions.html#dimensions)

In [ ]:
ds.Conventions

In [ ]:
ds.u10.dims

In [ ]:
my_ds.dims

The final step is to write the dataset to a dfs2 file. The Dataset already contains all the information needed.

In [ ]:
my_ds.to_dfs("era5_aoi.dfs2")


Screenshot of U 10m from MIKE Zero

![](images/era5_u10_dfs2.png)

## Unstructured grid forcing

E.g. roughness map

In [ ]:
msh = mikeio.Mesh("data/FakeLake.dfsu")
msh.plot();

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

z = np.linspace(0,-30)
manning = 32 + (40 - 32)*(1 - np.exp(z/10))

plt.plot(z, manning)
plt.xlim(0,-30)
plt.ylabel("Manning")
plt.xlabel("Depth (m)")
plt.title("This is only an example...")

Extract the Z values in each element.

In [ ]:
ze = msh.element_coordinates[:,2]

Create a new array of manning numbers based on the depth. This is only a hypothetical example of a data transformation based on the data in the file, not a best practise for bed friction.

In [ ]:
me = 32 + (40 - 32)*(1 - np.exp(ze/10))

The primary use case of DFS files is for time-varying data, so in order to use this time-independent data, we have to introduce a new axis, a time dimension.

One simple way to do this is to use the numpy function `atleast_2d`.

In [ ]:
me2d = np.atleast_2d(me) # Introduce time axis
me2d.shape

Now the array is a 2d array

In [ ]:
from mikeio.eum import ItemInfo

my_ds = mikeio.Dataset(data=[me2d],
                time="2000-1-1", # For static data, this is not used by MIKE 21
                items=[ItemInfo("Roughness", EUMType.Mannings_M)],
                geometry=msh.geometry)
my_ds

In [ ]:
my_ds.Roughness.plot();

In [ ]:
my_ds.to_dfs("manning.dfsu")

In [ ]:
dfs = mikeio.open("manning.dfsu")
dfs

In [ ]:
ds = dfs.read()
ds["Roughness"].plot(title="Manning");